In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
directory_in = './file/2.part'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：543



,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$171.79""}",5756,8304960,MELLING;30250S;5756;8304960
1,2,Engine,Timing Chain & Component Kit,MELLING,30250SX,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=11...,"{""0"": ""Does Not Include Sprockets""}","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$135.79""}",5756,11158045,MELLING;30250SX;5756;11158045
2,3,Engine,Timing Chain & Component Kit,MELLING,30420S,,,"https://www.rockauto.com/en/parts/melling,3042...","{""0"": ""https://www.rockauto.com/info/583/3-042...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$139.79""}",5756,8304968,MELLING;30420S;5756;8304968
3,4,Engine,Timing Chain & Component Kit,MELLING,30430S,,,"https://www.rockauto.com/en/parts/melling,3043...","{""0"": ""https://www.rockauto.com/info/583/3-043...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$188.79""}",5756,8304972,MELLING;30430S;5756;8304972
4,5,Engine,Timing Chain & Component Kit,MELLING,30460S,,,"https://www.rockauto.com/en/parts/melling,3046...","{""0"": ""https://www.rockauto.com/info/583/3-046...",https://www.rockauto.com/en/moreinfo.php?pk=83...,"{""0"": ""Does Not Include Sprockets""}","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$223.79""}",5756,8304976,MELLING;30460S;5756;8304976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,539,Engine,Timing Chain & Component Kit,MELLING,TOPK1014,,,"https://www.rockauto.com/en/parts/melling,TOPK...","{""0"": ""https://www.rockauto.com/info/583/TOPK-...",https://www.rockauto.com/en/moreinfo.php?pk=15...,"{""0"": ""Includes Standard Volume/Pressure Oil P...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$1,106.79""}",5756,15383849,MELLING;TOPK1014;5756;15383849
539,540,Engine,Timing Chain & Component Kit,MELLING,TOPK1015,,,"https://www.rockauto.com/en/parts/melling,TOPK...","{""0"": ""https://www.rockauto.com/info/583/TOPK-...",https://www.rockauto.com/en/moreinfo.php?pk=15...,"{""0"": ""Includes Oil Pump, Variable Valve Timin...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$303.79""}",5756,15583693,MELLING;TOPK1015;5756;15583693
540,541,Engine,Timing Chain & Component Kit,MELLING,TOPK1015HV,,,"https://www.rockauto.com/en/parts/melling,TOPK...","{""0"": ""https://www.rockauto.com/info/583/TOPK-...",https://www.rockauto.com/en/moreinfo.php?pk=15...,"{""0"": ""Includes High Volume Oil Pump, Variable...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$333.79""}",5756,15583697,MELLING;TOPK1015HV;5756;15583697
541,542,Engine,Timing Chain & Component Kit,MELLING,TOPK1016,,,"https://www.rockauto.com/en/parts/melling,TOPK...",{},https://www.rockauto.com/en/moreinfo.php?pk=16...,"{""0"": ""Includes Oil Pump, Variable Valve Timin...","{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$429.79""}",5756,16272549,MELLING;TOPK1016;5756;16272549


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                              1
 Group                                                      Engine
 Type                                 Timing Chain & Component Kit
 Manufacturer                                              MELLING
 Part Number                                                30250S
 OEM                                                              
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/melling,3025...
 Json_Src        {"0": "https://www.rockauto.com/info/583/3-025...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 Json_Note 1                                                    {}
 Json_Note 2       {"0": "Category: Timing Chain & Component Kit"}
 Choose                                                           
 Json_Price                                       {"0": "$171.79"}
 Type Code                                        

In [5]:
import time

data = {'func': 'getbuyersguide',
        'payload': json.dumps({'partData': {'listing_data_essential': {'parttype': crawler_series['Type Code'],
                                                                       'partkey': crawler_series['Part Code']},
                                            'listing_data_supplemental': {'partnumber': crawler_series['Part Number'],
                                                                          'catalogname': crawler_series['Manufacturer']}}}),
        'api_json_request': '1'}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if 'redirect_to_url' not in dict_ and dict_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'buyersguidepieces': {'body': '<div class="buyersguide-nested"><div style="padding: .5em;"><table class="nobmp" style="width: 100%;"><tr><td class="altrow-a-0">MAZDA</td><td class="altrow-a-0">3</td><td class="altrow-a-0">2004-2007</td></tr><tr><td class="altrow-a-1">MAZDA</td><td class="altrow-a-1">5</td><td class="altrow-a-1">2006-2007</td></tr><tr><td class="altrow-a-0">MAZDA</td><td class="altrow-a-0">6</td><td class="altrow-a-0">2003-2007</td></tr></table><div>Please refer to catalog for application details.</div></div></div>',
  'title': "Buyer's Guide : MELLING 30250S Timing Chain & Component Kit"},
 'groupindex': None,
 'collected_javascript': '/**UQ:C940683E**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL80\\/oQB6RvzNfa5dcHvATqXAMxw5whzaDEdxk+27zeij67BjKru6WqTdNqG6tiEsD1EZhbTv0XForkbYp+nLaIKxDXfb4DjpqS2Ysll2uy4rSPuo8L6J2H3Xjh8EpP3xTEDk=";\n'}

In [6]:
soup = BeautifulSoup(dict_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       3
      </td>
      <td class="altrow-a-0">
       2004-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       MAZDA
      </td>
      <td class="altrow-a-1">
       5
      </td>
      <td class="altrow-a-1">
       2006-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       6
      </td>
      <td class="altrow-a-0">
       2003-2007
      </td>
     </tr>
    </table>
    <div>
     Please refer to catalog for application details.
    </div>
   </div>
  </div>
 </body>
</html>



In [7]:
list_tr = html.xpath('//tr')

list_tr

[<Element tr at 0x10779e580>,
 <Element tr at 0x1200e2d00>,
 <Element tr at 0x1200e15c0>]

In [8]:
not list_tr and not (html.xpath('//p/text()') and html.xpath('//p/text()')[0].strip() == 'No applications found.')

False

In [9]:
list_dict_correct_temp = []
for i, tr in enumerate(list_tr):
    list_text = [text.strip() for text in tr.xpath('./td/text()')]

    if len(list_text) == 3:
        list_year = list_text[2].split('-')
        year_begin, year_end = int(list_year[0]), int(list_year[-1])
        list_dict_correct_temp_temp = [{'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC'],
                                        'Vehicle No': i + 1,
                                        'Make': list_text[0],
                                        'Model': list_text[1],
                                        'Year': year} for year in range(year_begin, year_end + 1)]
    elif len(list_text) == 2:
        list_year = list_text[1].split('-')
        year_begin, year_end = int(list_year[0]), int(list_year[-1])
        list_dict_correct_temp_temp = [{'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC'],
                                        'Vehicle No': i + 1,
                                        'Make': list_text[0],
                                        'Year': year} for year in range(year_begin, year_end + 1)]
    else:
        print('fuck')

    list_dict_correct_temp.extend(list_dict_correct_temp_temp)

list_dict_correct_temp

[{'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2004},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2005},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2006},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2007},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 2,
  'Make': 'MAZDA',
  'Model': '5',
  'Year': 2006},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 2,
  'Make': 'MAZDA',
  'Model': '5',
  'Year': 2007},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 3,
  'Make': 'MAZDA',
  'Model': '6',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 3,
  'Make': 'MAZDA',
  'Model': '6',
  'Year': 2004},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8

In [10]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2004},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2005},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2006},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 1,
  'Make': 'MAZDA',
  'Model': '3',
  'Year': 2007},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 2,
  'Make': 'MAZDA',
  'Model': '5',
  'Year': 2006},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 2,
  'Make': 'MAZDA',
  'Model': '5',
  'Year': 2007},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 3,
  'Make': 'MAZDA',
  'Model': '6',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8304960',
  'Vehicle No': 3,
  'Make': 'MAZDA',
  'Model': '6',
  'Year': 2004},
 {'JOIN_MPNTCPC': 'MELLING;30250S;5756;8

In [11]:
crawler_status = True

crawler_status

True

In [12]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                              1
 Group                                                      Engine
 Type                                 Timing Chain & Component Kit
 Manufacturer                                              MELLING
 Part Number                                                30250S
 OEM                                                              
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/melling,3025...
 Json_Src        {"0": "https://www.rockauto.com/info/583/3-025...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=83...
 Json_Note 1                                                    {}
 Json_Note 2       {"0": "Category: Timing Chain & Component Kit"}
 Choose                                                           
 Json_Price                                       {"0": "$171.79"}
 Type Code                                                    

In [13]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['JOIN_MPNTCPC', 'Vehicle No', 'Year'],
                                                             ascending=[True, True, False],
                                                             ignore_index=True)
    df_correct.to_excel('./test_vehicle.xlsx', index=False)

df_correct

,JOIN_MPNTCPC,Vehicle No,Make,Model,Year
0,MELLING;30250S;5756;8304960,1,MAZDA,3,2007
1,MELLING;30250S;5756;8304960,1,MAZDA,3,2006
2,MELLING;30250S;5756;8304960,1,MAZDA,3,2005
3,MELLING;30250S;5756;8304960,1,MAZDA,3,2004
4,MELLING;30250S;5756;8304960,2,MAZDA,5,2007
5,MELLING;30250S;5756;8304960,2,MAZDA,5,2006
6,MELLING;30250S;5756;8304960,3,MAZDA,6,2007
7,MELLING;30250S;5756;8304960,3,MAZDA,6,2006
8,MELLING;30250S;5756;8304960,3,MAZDA,6,2005
9,MELLING;30250S;5756;8304960,3,MAZDA,6,2004


In [14]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_vehicle_error.xlsx', index=False)

df_error

,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC,Check_Url
0,1,Engine,Timing Chain & Component Kit,MELLING,30250S,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,{},"{""0"": ""Category: Timing Chain & Component Kit""}",,"{""0"": ""$171.79""}",5756,8304960,MELLING;30250S;5756;8304960,"https://www.rockauto.com/en/parts/melling,3025..."
